In [1]:
from gensim.utils import tokenize
from langdetect import detect
import string
from itertools import islice
import numpy as np
from scipy.sparse import csr_matrix
import smart_open
import faiss

import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.ml.linalg import Vectors, VectorUDT

import pickle

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1567779103968_0001,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
model_path = "s3://onai-ml-dev-eu-west-1/company2vec/model/nmf"
word2id_path = "s3://onai-ml-dev-eu-west-1/company2vec/common"
data_path = "s3://onai-ml-dev-eu-west-1/company2vec/data_nmf"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
word2id = {}
id2word = {}
with smart_open.open(f"{word2id_path}/bow/word2id.csv", "r") as f:
    for line in f:
        word,idd = line.strip().split(",")
        word2id[word] = int(idd)
        id2word[int(idd)] = word

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
with smart_open.open(f"{model_path}/H/mat.np", "rb") as f:
    H = np.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
len(word2id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

42829

In [6]:
with smart_open.open(f"{model_path}/X/X_nmf.np", "rb") as f:
    X_nmf = np.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
with smart_open.open(f"{model_path}/X/company_info.np", "rb") as f:
    description = np.load(f, allow_pickle=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
n,d = X_nmf.shape
n,d

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3381364, 200)

In [9]:
index = faiss.IndexFlatL2(d)
index.add(X_nmf)
print(index.ntotal)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3381364

In [10]:
company2id = {}
for i,desc in enumerate(description):
    company2id[desc[0]] = i

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
def query_peers(query_company, result_size=100):
    query_vector = X_nmf[np.newaxis, company2id[query_company]]
    
    desc_info = description[company2id[query_company]]
    industry = desc_info[-2]
    country = desc_info[-3]
    
    D, I = index.search(query_vector, result_size)
    
    ret = []
    added = set()

#     for i,el in enumerate(I[0]):
#         if description[el,-2] == industry and description[el,-3] == country:
#             el = (D[0, i], description[el])
#             if repr(el) not in added:
#                 ret.append(el)
#                 added.add(repr(el))
#         if len(ret) >= result_size:
#             break

#     # TODO: once we have country embeddings, order by them
#     if len(ret) < result_size:
#         for i,el in enumerate(I[0]):
#             if description[el,-2] == industry:
#                 el = (D[0, i], description[el])
#                 if repr(el) not in added:
#                     ret.append(el)
#                     added.add(repr(el))
#             if len(ret) >= result_size:
#                 break

#     # TODO: once we have industry class embeddings, order by them
#     if len(ret) < result_size:
#         for i,el in enumerate(I[0]):
#             if description[el,-3] == country:
#                 el = (D[0, i], description[el])
#                 if repr(el) not in added:
#                     ret.append(el)
#                     added.add(repr(el))
#             if len(ret) >= result_size:
#                 break

    if len(ret) < result_size:
        for i,el in enumerate(I[0]):
            el = (D[0, i], description[el])
            if repr(el) not in added:
                ret.append(el)
                added.add(repr(el))
            if len(ret) >= result_size:
                break
                
    return ret


def find_peer_in_result(results, peer):
    for i, company in enumerate(results):
        if peer == company:
            return i+1

    return -1


def add_ranks_in_tops(ranks, counts):
    for rank in ranks:
        if 0 < rank <= 10:
            counts[0] += 1
        elif 10 < rank <= 20:
            counts[1] += 1
        elif 20 < rank <= 50:
            counts[2] += 1
        elif 50 < rank <= 100:
            counts[3] += 1
        elif 100 < rank:
            counts[4] += 1
        else:
            counts[5] += 1
    return counts

tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
for el in query_peers(874652):
    print(el)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.0, array([874652, 'Tata Motors Limited',
       'Tata Motors Limited designs, develops, manufactures, and sells a range of automotive vehicles. It offers cars, sports utility vehicles, trucks, buses, and defence vehicles, as well as related spare parts and accessories. The company also provides information technology services, machine tools, and factory automation services; and vehicle financing services. It offers its products under the Tata, Daewoo, Fiat, Jaguar, and Land Rover brands. Tata Motors Limited operates in India, the United States, the United Kingdom, rest of Europe, China, and internationally. The company was formerly known as Tata Engineering and Locomotive Company Limited and changed its name to Tata Motors Limited in July 2003. Tata Motors Limited was founded in 1945 and is headquartered in Mumbai, India.',
       'India', '', 40886.39353899557], dtype=object))
(0.40671325, array([667151, 'Suzuki Motor Corporation',
       'Suzuki Motor Corporation manufactures and 

In [22]:
for el in query_peers(813607, 500):
    print(el)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.0, array([813607, 'América Móvil, S.A.B. de C.V.',
       'América Móvil, S.A.B. de C.V. provides telecommunications services in Latin America, the United States, the Caribbean, and Europe. The company offers wireless and fixed voice services, including airtime, local, domestic, and international long-distance services; and network interconnection services. It also provides data services, such as Internet access, messaging, and other wireless entertainment and corporate services; data transmission, email services, instant messaging, content streaming, and interactive applications; and wireless security services, mobile payment solutions, machine-to-machine services, mobile banking, virtual private network services, and video calls and personal communications services. In addition, the company offers data center, data administration, and hosting services to residential and corporate clients, as well as residential broadband services; IT solutions for small businesses and large corpor

In [23]:
public_labels = {
    2444485: [9429145, 93196, 9870522, 7914436],
    645782: [380011, 392154, 5523392],
    875173: [237655379, 931146, 418171],
    813607: [100231, 357076],
    380011: [357076, 324490, 93339],
    882889: [127202, 5478606, 1025390, 645782],
    46329052: [189096, 915379, 46895276, 877008, 325290136, 20524024, 271958947, 21852987, 26363560, 110104150],
    106335: [319676, 377732, 61206100],
    877769: [874042, 780678, 953488, 883809, 875295, 874186, 874119],
    34049884: [882155, 30428758, 315394, 23037669, 27561],
    254287477: [883300, 30614595],
    5600108: [285880, 5433540, 878697, 35650, 688262, 226852452],
    876031: [410182, 874470, 874191, 879732, 5395336],
    883752: [880697, 65340486, 26320074, 883327, 1034090, 257501324],
    5920885: [1494039, 268074105, 34534627, 20385800, 23000545, 124640],
    628413: [272054403, 91192, 309779, 140283, 138644],
    364040: [381388, 184945, 874170, 42751952, 874183, 314896, 5126590, 841504],
    257501324: [35000, 47320264, 253748612, 85076655, 32053, 12144785, 8186273, 9934160, 557267859],
    695204: [35303, 274561, 683719, 370857, 561001, 874022, 387473, 394038, 8274485, 12188205],
    30614595: [883300, 254287477, 9956099, 380011, 27868703, 2386697, 126857],
    28224119: [26824144, 35023689, 386639, 393661],
    32449506: [875260, 27169270, 5629762, 26014489, 286119],
    233324810: [874864, 159230, 27860587, 35806, 876981],
    879554: [5487000, 236715563, 412090459, 875192, 278679, 180871, 22516334, 30274893, 5478907],
    5580060: [118474533, 1779941, 265154, 10405454],
    23335317: [7885406, 277444, 278933, 8983678, 874143, 409119],
    381865672: [874842, 410366, 873649, 275789, 882473],
    937352: [876758, 879422, 128861678, 6461781, 1859063],
    874119: [881803, 875849, 231533, 877769, 780678, 953488, 875295, 874042, 775001, 874186],
    680934: [135398, 882299, 668578, 4481676, 32012],
    2248076: [141249, 4975204, 98876, 21828553],
    3606442: [882547, 4509042, 20703565, 7435035, 94799, 288033, 359868],
    877235: [295170, 175265, 874520, 410366, 873649, 874977, 167945, 8090046],
    84148802: [275789, 30339992, 5533238, 5718736],
    5523392: [645782, 11809880, 1353107, 962864],
    413744: [409932, 875491, 109303666, 91638],
    314896: [330589, 34768, 184945],
    5126590: [874855, 631781, 364040, 831357, 874170],
    377732: [319676, 106772, 106335, 704634, 320105, 874828, 873861, 1519242, 533853947],
    874652: [377732, 319676, 106772, 704634, 312375, 278933, 874828, 4863668]
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
for public_label,peers in public_labels.items():
    query_id = public_label
    results = [el[1][0] for el in query_peers(query_id)]
    ranks = []
    
    for peer in peers:
        ranks.append(find_peer_in_result(results, peer))
        ranks_default.append(find_peer_in_result(results, peer))

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, -1, -1, -1, -1]

[645782, -1, -1, -1]

[875173, -1, -1, -1]

[813607, -1, 2]

[380011, -1, 94, -1]

[882889, 84, -1, -1, -1]

[46329052, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

[106335, -1, -1, 3]

[877769, -1, 74, -1, -1, -1, -1, -1]

[34049884, 21, -1, -1, -1, -1]

[254287477, -1, -1]

[5600108, -1, -1, -1, -1, -1, -1]

[876031, -1, -1, -1, 17, -1]

[883752, -1, -1, -1, -1, -1, -1]

[5920885, -1, -1, -1, -1, -1, -1]

[628413, -1, -1, -1, -1, -1]

[364040, 79, -1, -1, -1, 26, -1, -1, -1]

[257501324, -1, -1, -1, -1, -1, -1, -1, -1, -1]

[695204, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1]

[30614595, -1, -1, -1, -1, 67, -1, -1]

[28224119, -1, -1, -1, -1]

[32449506, -1, -1, -1, -1, -1]

[233324810, -1, -1, -1, -1, -1]

[879554, -1, -1, -1, -1, -1, -1, -1, -1, -1]

[5580060, -1, -1, -1, -1]

[23335317, -1, -1, -1, -1, -1, -1]

[381865672, -1, -1, -1, -1, -1]

[937352, -1, -1, -1, -1, -1]

[874119, -1, 3, -1, -1, -1, -1, -1, -1, 63, -1]

[680934, 32, 17, 3, 24, -1]

[2248076, -1,

In [25]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
for public_label,peers in public_labels.items():
    query_id = public_label
    results = [el[1][0] for el in query_peers(query_id, 500)]
    ranks = []
    
    for peer in peers:
        ranks.append(find_peer_in_result(results, peer))
        ranks_default.append(find_peer_in_result(results, peer))

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 313, -1, -1, -1]

[645782, -1, -1, -1]

[875173, -1, -1, 143]

[813607, -1, 2]

[380011, 103, 94, 191]

[882889, 84, -1, -1, 232]

[46329052, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

[106335, 249, 308, 3]

[877769, 391, 74, 211, -1, 241, -1, -1]

[34049884, 21, -1, -1, -1, -1]

[254287477, -1, -1]

[5600108, -1, -1, -1, -1, -1, -1]

[876031, -1, -1, -1, 17, -1]

[883752, -1, -1, -1, -1, -1, -1]

[5920885, -1, -1, -1, -1, -1, -1]

[628413, -1, -1, -1, -1, -1]

[364040, 79, 309, -1, 456, 26, -1, -1, 174]

[257501324, -1, 294, -1, -1, -1, -1, -1, -1, -1]

[695204, 3, 235, -1, -1, -1, 137, -1, -1, -1, -1]

[30614595, -1, -1, -1, -1, 67, -1, -1]

[28224119, -1, -1, -1, -1]

[32449506, -1, -1, -1, -1, -1]

[233324810, -1, -1, -1, -1, -1]

[879554, -1, -1, -1, -1, -1, -1, -1, -1, -1]

[5580060, -1, -1, -1, -1]

[23335317, -1, -1, -1, -1, -1, -1]

[381865672, -1, -1, -1, -1, -1]

[937352, -1, -1, -1, -1, -1]

[874119, -1, 3, 387, -1, -1, -1, -1, -1, 63, -1]

[680934, 32, 17, 3, 24, -